<a href="https://colab.research.google.com/github/mchan000/CLIPDetect/blob/main/CLIPAIDetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("hello")

hello
